<a href="https://colab.research.google.com/github/KayalvizhiT513/Tech-Case-Studies/blob/main/Prisoner_Wine_Bottle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------------------
# GF(16) arithmetic
# ------------------------------

EXP = [0] * 30    # exp table
LOG = [0] * 16    # log table

def init_gf16():
    poly = 0b10011   # x^4 + x + 1
    x = 1
    for i in range(15):
        EXP[i] = x
        LOG[x] = i
        x <<= 1
        if x & 0b10000:   # if degree >= 4
            x ^= poly
    for i in range(15, 30):
        EXP[i] = EXP[i - 15]

def gf16_add(a, b):
    return a ^ b

def gf16_mul(a, b):
    if a == 0 or b == 0:
        return 0
    return EXP[(LOG[a] + LOG[b]) % 15]

init_gf16()


In [ ]:
# ------------------------------
# Build RS-like q-ary codewords
# ------------------------------

def rs_codewords(n):
    """
    Generate n q-ary codewords over GF(16), length L=3.
    Codewords = (f(1), f(2), f(4))
    where f(x) = a0 + a1 x + a2 x^2
    """
    codewords = []
    for i in range(n):
        a0 = i % 16
        a1 = (i >> 4) % 16
        a2 = (i >> 8) % 16

        w = []
        for x in [1, 2, 4]:
            y = gf16_add(a0,
                gf16_add(gf16_mul(a1, x),
                         gf16_mul(a2, gf16_mul(x, x))))
            w.append(y)
        codewords.append(w)
    return codewords


In [ ]:
# ------------------------------
# Expand q-ary to binary matrix
# ------------------------------

def expand_binary(codewords, q=16):
    cols = []
    for w in codewords:
        col = []
        for sym in w:
            block = [0] * q
            block[sym] = 1
            col.extend(block)
        cols.append(col)

    # convert columns → row-major matrix
    rows = list(map(list, zip(*cols)))   # m x n
    return rows


In [ ]:
# ------------------------------
# Full Kautz–Singleton matrix
# ------------------------------

def ks_matrix_3_disjunct(n=1000):
    q = 16       # alphabet size
    L = 3        # codeword length
    m = q * L    # total rows = 48

    codewords = rs_codewords(n)
    rows = expand_binary(codewords, q)
    return rows   # 48 x n binary matrix


In [ ]:
rows = ks_matrix_3_disjunct(1000)
print("Matrix shape:", len(rows), "x", len(rows[0]))


Matrix shape: 48 x 1000


In [ ]:
def test_poison(rows, pois):
    m = len(rows)
    death = [0] * m
    for c in pois:
        for r in range(m):
            if rows[r][c] == 1:
                death[r] = 1
    return death


In [ ]:
def fast_decode(rows, death_pattern):
    m = len(rows)
    n = len(rows[0])
    result = []
    for col in range(n):
        ok = True
        for r in range(m):
            if rows[r][col] == 1 and death_pattern[r] == 0:
                ok = False
                break
        if ok:
            result.append(col)
    return result


In [ ]:
rows = ks_matrix_3_disjunct(1000)

# true poison
true_poison = [10, 57, 888]

# simulate
death = test_poison(rows, true_poison)

# decode
found = fast_decode(rows, death)

print("Decoded:", found)


Decoded: [10, 57, 888]


In [ ]:
# ------------------------------
# GF(16) arithmetic
# ------------------------------

EXP = [0] * 30    # exp table
LOG = [0] * 16    # log table

def init_gf16():
    poly = 0b10011   # x^4 + x + 1
    x = 1
    for i in range(15):
        EXP[i] = x
        LOG[x] = i
        x <<= 1
        if x & 0b10000:   # if degree >= 4
            x ^= poly
    for i in range(15, 30):
        EXP[i] = EXP[i - 15]

def gf16_add(a, b):
    return a ^ b

def gf16_mul(a, b):
    if a == 0 or b == 0:
        return 0
    return EXP[(LOG[a] + LOG[b]) % 15]

init_gf16()


# ------------------------------
# Build RS-like q-ary codewords
# ------------------------------

def rs_codewords(n):
    """
    Generate n q-ary codewords over GF(16), length L=3.
    Codewords = (f(1), f(2), f(4))
    where f(x) = a0 + a1 x + a2 x^2
    """
    codewords = []
    for i in range(n):
        a0 = i % 16
        a1 = (i >> 4) % 16
        a2 = (i >> 8) % 16

        w = []
        for x in [1, 2, 4]:
            y = gf16_add(a0,
                gf16_add(gf16_mul(a1, x),
                         gf16_mul(a2, gf16_mul(x, x))))
            w.append(y)
        codewords.append(w)
    return codewords


# ------------------------------
# Expand q-ary to binary matrix
# ------------------------------

def expand_binary(codewords, q=16):
    cols = []
    for w in codewords:
        col = []
        for sym in w:
            block = [0] * q
            block[sym] = 1
            col.extend(block)
        cols.append(col)

    # convert columns → row-major matrix
    rows = list(map(list, zip(*cols)))   # m x n
    return rows


# ------------------------------
# Full Kautz–Singleton matrix
# ------------------------------

def ks_matrix_3_disjunct(n=1000):
    q = 16       # alphabet size
    L = 3        # codeword length
    m = q * L    # total rows = 48

    codewords = rs_codewords(n)
    rows = expand_binary(codewords, q)
    return rows   # 48 x n binary matrix


rows = ks_matrix_3_disjunct(1000)
print("Matrix shape:", len(rows), "x", len(rows[0]))


def test_poison(rows, pois):
    m = len(rows)
    death = [0] * m
    for c in pois:
        for r in range(m):
            if rows[r][c] == 1:
                death[r] = 1
    return death


def fast_decode(rows, death_pattern):
    m = len(rows)
    n = len(rows[0])
    result = []
    for col in range(n):
        ok = True
        for r in range(m):
            if rows[r][col] == 1 and death_pattern[r] == 0:
                ok = False
                break
        if ok:
            result.append(col)
    return result


rows = ks_matrix_3_disjunct(1000)

# true poison
true_poison = [10, 57, 888]

# simulate
death = test_poison(rows, true_poison)

# decode
found = fast_decode(rows, death)

print("Decoded:", found)


